In [2]:
# avaliacao_processor.py
import pandas as pd
import yaml
import logging
from typing import Dict, List, Tuple
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from pprint import pprint

class AvaliacaoDesempenhoProcessor:
    def __init__(self):
        self.logger = logging.getLogger(__name__)
        logging.basicConfig(level=logging.INFO, 
                          format='%(asctime)s - %(levelname)s - %(message)s')

    def check_files(self, yaml_path: str, excel_path: str) -> bool:
        """Verifica se os arquivos necessários existem."""
        yaml_file = Path(yaml_path)
        excel_file = Path(excel_path)
        
        if not yaml_file.exists():
            self.logger.error(f"Arquivo de configuração não encontrado: {yaml_path}")
            return False
            
        if not excel_file.exists():
            self.logger.error(f"Arquivo de dados não encontrado: {excel_path}")
            return False
            
        return True

    def load_config(self, yaml_path: str) -> Dict:
        """Carrega configuração do YAML."""
        try:
            with open(yaml_path, 'r', encoding='utf-8') as file:
                config = yaml.safe_load(file)
            self.logger.info(f"Configuração YAML carregada com sucesso")
            # Debug: mostrar estrutura do YAML
            self.logger.info("Estrutura do YAML:")
            pprint(config)
            return config
        except Exception as e:
            self.logger.error(f"Erro ao carregar YAML: {str(e)}")
            raise

    def load_data(self, excel_path: str, sheet_name: str = None) -> pd.DataFrame:
        """Carrega dados do Excel."""
        try:
            df = pd.read_excel(excel_path, sheet_name=sheet_name)
            
            # Debug: mostrar colunas carregadas
            self.logger.info("Colunas carregadas:")
            pprint(list(df.columns))
            
            # Remover colunas conforme configuração
            if hasattr(self, 'config') and 'csv_processing' in self.config:
                discard_columns = self.config['csv_processing'].get('discard_columns', [])
                for col in discard_columns:
                    if col in df.columns:
                        df = df.drop(columns=[col])
            
            self.logger.info(f"Dados carregados: {len(df)} linhas")
            return df
        except Exception as e:
            self.logger.error(f"Erro ao carregar Excel: {str(e)}")
            raise

    def process_competencias(self, df: pd.DataFrame, config: Dict) -> Dict:
        """Processa as competências e suas avaliações."""
        results = {}
        
        try:
            # Debug: verificar se temos acesso às competências
            if 'avaliacao' not in config:
                self.logger.error("Chave 'avaliacao' não encontrada no config")
                raise KeyError("Chave 'avaliacao' não encontrada")
                
            if 'competencias' not in config['avaliacao']:
                self.logger.error("Chave 'competencias' não encontrada em config['avaliacao']")
                raise KeyError("Chave 'competencias' não encontrada")
            
            # Processar cada competência
            for competencia in config['avaliacao']['competencias']:
                nome = competencia['nome']
                self.logger.info(f"Processando competência: {nome}")
                
                results[nome] = {
                    'objetivas': {},
                    'subjetivas': {},
                    'estatisticas': {},
                    'descricao': competencia['descricao']
                }
                
                # Debug: mostrar perguntas objetivas disponíveis
                self.logger.info(f"Perguntas objetivas para {nome}:")
                pprint(competencia['perguntas']['objetivas'])
                
                # Processar perguntas objetivas
                for pergunta in competencia['perguntas']['objetivas']:
                    if pergunta in df.columns:
                        stats = df[pergunta].describe()
                        results[nome]['objetivas'][pergunta] = {
                            'media': stats['mean'],
                            'mediana': df[pergunta].median(),
                            'desvio_padrao': stats['std'],
                            'min': stats['min'],
                            'max': stats['max'],
                            'contagem': stats['count']
                        }
                    else:
                        self.logger.warning(f"Pergunta não encontrada no DataFrame: {pergunta}")
                
                # Processar perguntas subjetivas
                for pergunta in competencia['perguntas']['subjetivas']:
                    if pergunta in df.columns:
                        respostas = df[pergunta].dropna().tolist()
                        results[nome]['subjetivas'][pergunta] = {
                            'respostas': respostas,
                            'total_respostas': len(respostas)
                        }
                    else:
                        self.logger.warning(f"Pergunta subjetiva não encontrada: {pergunta}")
                
                # Estatísticas gerais da competência
                perguntas_obj = [p for p in competencia['perguntas']['objetivas'] if p in df.columns]
                if perguntas_obj:
                    media_geral = df[perguntas_obj].mean().mean()
                    results[nome]['estatisticas'] = {
                        'media_geral': media_geral,
                        'total_avaliacoes': len(df),
                        'total_perguntas': len(perguntas_obj),
                        'avaliadores': competencia['avaliadores']
                    }
            
            self.logger.info("Processamento das competências concluído")
            return results
            
        except Exception as e:
            self.logger.error(f"Erro no processamento das competências: {str(e)}")
            self.logger.error("Estado atual do config:")
            pprint(config)
            raise

    # [resto do código permanece igual...]

def main():
    # Caminhos dos arquivos
    YAML_PATH = "competencias.yaml"
    EXCEL_PATH = "responses.xlsx"
    OUTPUT_PATH = "resultados_avaliacao_desempenho.xlsx"
    HEATMAPS_DIR = "heatmaps"
    
    # Criar instância do processador
    processor = AvaliacaoDesempenhoProcessor()
    
    try:
        # Verificar arquivos
        if not processor.check_files(YAML_PATH, EXCEL_PATH):
            print("Por favor, verifique se os arquivos necessários existem no diretório.")
            return
        
        # Carregar configuração e dados
        processor.config = processor.load_config(YAML_PATH)
        df = processor.load_data(EXCEL_PATH)
        
        # Debug: mostrar estrutura dos dados carregados
        print("\nEstrutura dos dados:")
        print("Colunas disponíveis:")
        pprint(list(df.columns))
        print("\nPrimeiras linhas:")
        print(df.head())
        
        # Processar avaliações
        results = processor.process_competencias(df, processor.config)
        
        print("\nProcessamento concluído com sucesso!")
        print("\nResumo das competências processadas:")
        for comp_nome, comp_data in results.items():
            print(f"\n{comp_nome}:")
            if 'estatisticas' in comp_data:
                stats = comp_data['estatisticas']
                print(f"  Média Geral: {stats.get('media_geral', 0):.2f}")
                print(f"  Total de Avaliações: {stats.get('total_avaliacoes', 0)}")
                print(f"  Total de Perguntas: {stats.get('total_perguntas', 0)}")
                print(f"  Avaliadores: {', '.join(stats.get('avaliadores', []))}")
        
    except Exception as e:
        print(f"Erro durante o processamento: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

2024-10-22 13:10:51,600 - INFO - Configuração YAML carregada com sucesso
2024-10-22 13:10:51,601 - INFO - Estrutura do YAML:
/home/roni-chittoni/ProjectsHub/GAPD/avaliacao_desempenho/venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
2024-10-22 13:10:51,642 - INFO - Colunas carregadas:
2024-10-22 13:10:51,642 - ERROR - Erro ao carregar Excel: 'dict' object has no attribute 'columns'


{'avaliacao': {'pessoa_avaliada': {'identificador_pergunta': 'Sua relacao com '
                                                             '{{nome}} e '
                                                             'como:'}},
 'categorias_avaliador': {'campo_identificacao': 'Tags',
                          'categorias': ['diretor',
                                         'colega gestor',
                                         'liderado',
                                         'autoavaliacao']},
 'competencias': [{'avaliadores': ['liderado', 'autoavaliacao'],
                   'descricao': 'Planeja e apoia o desenvolvimento das '
                                'habilidades de todas as pessoas de sua equipe '
                                'para que possam atender aos objetivos atuais '
                                'e futuros de seus PAs. A pessoa demonstra '
                                'essa competência: 1. Atribuindo '
                                'responsabilidades

Traceback (most recent call last):
  File "/tmp/ipykernel_20832/3199201602.py", line 164, in main
    df = processor.load_data(EXCEL_PATH)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_20832/3199201602.py", line 53, in load_data
    pprint(list(df.columns))
                ^^^^^^^^^^
AttributeError: 'dict' object has no attribute 'columns'
